In [2]:
import os
from dotenv import load_dotenv

In [3]:
# function t load all environment variables safely

def safe_set_env(var_name: str, default: str = ""):
    """Safely set os.environ with string default - No NoneType errors."""
    value = os.getenv(var_name, default)
    if value is None:
        value = default
    os.environ[var_name] = str(value)

# Usage - Zero crashes
load_dotenv()
safe_set_env("OPENAI_API_KEY", "your-openai-key-here")
safe_set_env("LANGCHAIN_ENDPOINT", "")
safe_set_env("LANGCHAIN_API_KEY", "")
safe_set_env("LANGCHAIN_PROJECT", "default-project")
os.environ["LANGSMITH_TRACING"] = "False"

print("🚀 Production env ready!")

🚀 Production env ready!


In [4]:
from typing import TypedDict, List
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END, ToolNode

# State definition
class NumberState(TypedDict):
    initial_number: int
    final_number: int
    messages: List[BaseMessage]

# Tools
def add(x: int) -> int:
    return x + 5

def square(x: int) -> int:
    return x * x

tools = {"add": add, "square": square}
tool_node = ToolNode(tools)

# Bind LLM with tools
llm = ChatOpenAI(model="gpt-4o-mini").bind_tools(tools)

# Node 1: add_node
def add_node(state: NumberState) -> dict:
    num = state["initial_number"]
    msg = HumanMessage(content=f"Add 5 to {num}. Use the correct tool.")
    result = llm.invoke([msg])
    return {
        "messages": state.get("messages", []) + [msg, result],
        "initial_number": state["initial_number"],
        "final_number": state["final_number"]  # will be updated by tool_node
    }

# Node 2: square_node
def square_node(state: NumberState) -> dict:
    num = state["final_number"]
    msg = HumanMessage(content=f"Square {num}. Use the correct tool.")
    result = llm.invoke([msg])
    return {
        "messages": state.get("messages", []) + [msg, result],
        "initial_number": state["initial_number"],
        "final_number": state["final_number"]  # will be updated by tool_node
    }

# Build graph
workflow = StateGraph(NumberState)
workflow.add_node("add_node", add_node)
workflow.add_node("square_node", square_node)
workflow.add_node("tool_node", tool_node)

workflow.set_entry_point("add_node")

# Edges
workflow.add_edge("add_node", "tool_node")       # add_node → tool_node
workflow.add_edge("tool_node", "square_node")    # tool_node → square_node
workflow.add_edge("square_node", "tool_node")    # square_node → tool_node
workflow.add_edge("tool_node", END)              # tool_node → END

app = workflow.compile()

# Run
state: NumberState = {
    "initial_number": 3,
    "final_number": 3,
    "messages": []
}

final_state = app.invoke(state)

print("Final State:", final_state)
for msg in final_state["messages"]:
    print(msg)


ImportError: cannot import name 'ToolNode' from 'langgraph.graph' (c:\Users\HP\anaconda3\envs\NLQ_TO_PYTHON_CHATBOT_ENV\Lib\site-packages\langgraph\graph\__init__.py)